In [1]:
import pandas as pd
import numpy as np
import psycopg2
from psycopg2 import extras


In [2]:
df_payslip_line = pd.read_excel("./input/2023-12-20_16-19-21_QDML1nU3.xlsx",header=[0, 1], index_col=0)
df_payslip_line = df_payslip_line.drop(df_payslip_line.columns[3:17], axis=1)
df_payslip_line = df_payslip_line.drop(df_payslip_line.columns[1], axis=1)
melted_df = pd.melt(df_payslip_line, id_vars=list(df_payslip_line.columns[0:2])).fillna(0)
columns = ['employee_name','employee_name_id' , 'name', 'code', 'amount']
# # Đổi tên cột của DataFrame melted_df
melted_df.columns = columns
melted_df

,employee_name,employee_name_id,name,code,amount
0,Nguyễn Đoàn Nhật Nam,163430-01,Lương CB Tháng trước,1,0.0
1,Lê Khương Bảo Trung Kiên,163706-01,Lương CB Tháng trước,1,0.0
2,Nguyễn Văn Trọng,163796-01,Lương CB Tháng trước,1,0.0
3,Hàn Quốc Việt,164313-01,Lương CB Tháng trước,1,0.0
4,Nguyễn Đoàn Nhật Nam,163430-01,Lương căn bản,2,10000000.0
...,...,...,...,...,...
507,Hàn Quốc Việt,164313-01,Ghi chú,600,0.0
508,Nguyễn Đoàn Nhật Nam,163430-01,Số ngày phép còn lại,Phep_,0.0
509,Lê Khương Bảo Trung Kiên,163706-01,Số ngày phép còn lại,Phep_,0.0
510,Nguyễn Văn Trọng,163796-01,Số ngày phép còn lại,Phep_,0.0


### load hr_employee

In [13]:
import psycopg2

# connect to PostgreSQL
dbname = 'main_220803'
user = 'odoo'
password = 'odoo'
host = '10.10.10.24'
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=5433)
cursor = conn.cursor()

sql_query = '''
    SELECT id, name, code
    FROM hr_employee
'''

# Execute the SQL query
cursor.execute(sql_query)

# Fetch all the results
data_hr_employee = cursor.fetchall()
data_hr_employee = pd.DataFrame(data_hr_employee, columns=['employee_id', 'e_name', 'e_code'])

cursor.close()
conn.close()

In [14]:
data_hr_employee

,employee_id,e_name,e_code
0,16837,Nguyễn Thiên Bình,123738-01
1,19197,Đỗ Thị Quyên,127253-01
2,35569,Nguyễn Hữu Đức,126573-02
3,61607,Nguyễn Thị Lâm,146096-01
4,25032,Hồ Văn Nghiên,120999-01
...,...,...,...
41126,57141,Lê Thị Thu Trang,142378-01
41127,51343,Lê Đình Thắng,136980-01
41128,57036,Nguyễn Thị Hương,142297-01
41129,59023,Nguyễn Thị Nhật Lệ,145033-01


### Load Hr_payslip

In [15]:
import psycopg2

# connect to PostgreSQL
dbname = 'main_220803'
user = 'odoo'
password = 'odoo'
host = '10.10.10.24'
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=5433)
cursor = conn.cursor()

sql_query = '''
    SELECT id, employee_id
    FROM hr_payslip
'''

# Execute the SQL query
cursor.execute(sql_query)

# Fetch all the results
data_hr_payslip = cursor.fetchall()
data_hr_payslip = pd.DataFrame(data_hr_payslip, columns=['payslip_id', 'employee_id'])

cursor.close()
conn.close()

In [16]:
data_hr_payslip

,payslip_id,employee_id
0,311698,26917
1,314692,26917
2,362407,26917
3,362621,26917
4,381263,26917
...,...,...
53459,500282,26428
53460,433013,26428
53461,433048,26428
53462,433116,26428


In [17]:
join_payslip_empl = pd.merge(data_hr_payslip, data_hr_employee, how='inner', on='employee_id')

In [18]:
final_result = pd.merge(melted_df, join_payslip_empl, how='left', left_on='employee_name_id', right_on='e_code')
data_payslip_line = final_result[['employee_id', 'name', 'code', 'amount', 'payslip_id']]
data_payslip_line

,employee_id,name,code,amount,payslip_id
0,NaN,Lương CB Tháng trước,1,0.0,NaN
1,NaN,Lương CB Tháng trước,1,0.0,NaN
2,NaN,Lương CB Tháng trước,1,0.0,NaN
3,NaN,Lương CB Tháng trước,1,0.0,NaN
4,NaN,Lương căn bản,2,10000000.0,NaN
...,...,...,...,...,...
507,NaN,Ghi chú,600,0.0,NaN
508,NaN,Số ngày phép còn lại,Phep_,0.0,NaN
509,NaN,Số ngày phép còn lại,Phep_,0.0,NaN
510,NaN,Số ngày phép còn lại,Phep_,0.0,NaN


In [19]:
data_payslip_line.dtypes

employee_id    float64
name            object
code            object
amount         float64
payslip_id     float64
dtype: object

In [20]:
data_types = {
            'employee_id':int,
            'name': str,
            'code': str,
            'amount': float,
            'payslip_id': int
        }


import pandas as pd

class ProcessHrPayslipLine:
    def __init__(self, df, data_types):
        self.employee_id = self.replace_values(df['employee_id'].to_list(), data_types["employee_id"])
        self.name = self.replace_values(df['name'].to_list(), data_types["name"])
        self.code = self.replace_values(df['code'].to_list(), data_types["code"])
        self.amount = self.replace_values(df['amount'].to_list(), data_types["amount"])
        self.payslip_id = self.replace_values(df['payslip_id'].to_list(), data_types["payslip_id"])


    def replace_values(self, list_data, data_type):
        if data_type == int:
            return [self.try_int_conversion(value) if isinstance(value, (int, float)) else None for value in list_data]
        elif data_type == float:
            return [value if isinstance(value, (int, float)) else None for value in list_data]
        elif data_type == bool:
            return [value if isinstance(value, bool) else None for value in list_data]
        elif data_type == str:
            processed_values = []

            for value in list_data:
                if str(value) != 'nan' and len(str(value).split("."))>1:
                    if(str(value).split(".")[1] == '0'):
                        processed_values.append(str(value).split(".")[0])
                    else:
                        processed_values.append(str(value))
                elif str(value) == 'nan':
                    processed_values.append('')
                else:
                    processed_values.append(str(value))
            return processed_values
        elif data_type == 'datetime64[ns]':
            return [value if str(value) != 'nan' else None for value in list_data]
        else:
            return list_data

    def try_int_conversion(self, value):
        try:
            return int(value)
        except (ValueError, TypeError):
            return None

    def to_numpy(self):
        return np.array([
            self.employee_id,                    
            self.name,     
            self.code,     
            self.amount,
            self.payslip_id
        ], dtype=object).T



In [21]:
data = ProcessHrPayslipLine(data_payslip_line, data_types)
data.employee_id[0] = 73996
data.employee_id[1] = 529
data.payslip_id[0] = 527039
data.payslip_id[1] = 527040

In [22]:
value = data.to_numpy()[0:2]
value

array([[73996, 'Lương CB Tháng trước', '1', 0.0, 527039],
       [529, 'Lương CB Tháng trước', '1', 0.0, 527040]], dtype=object)

In [23]:
import psycopg2
from psycopg2 import extras

# connect to PostgreSQL
dbname = 'main_220803'
user = 'odoo'
password = 'odoo'
host = '10.10.10.24'
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=5433)
cursor = conn.cursor()

# Define the SQL query for bulk insert
sql_query = '''
    INSERT INTO hr_payslip_line (
        employee_id, name, code, amount, slip_id
    )
    VALUES %s
    RETURNING ID
'''

# Execute the SQL query with the values from the NumPy array
extras.execute_values(cursor, sql_query, value.tolist(), page_size=len(value))

# Commit the transaction
conn.commit()

# Fetch the returned IDs
returned_ids = cursor.fetchall()

# Print or use the returned IDs as needed
print("Returned IDs:", returned_ids)

cursor.close()
conn.close()

NotNullViolation: null value in column "salary_rule_id" of relation "hr_payslip_line" violates not-null constraint
DETAIL:  Failing row contains (115530416, 527039, null, 73996, null, null, 0.0, null, null, Lương CB Tháng trước, 1, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, unprocessed).
